## Imports and Data Download

In [1]:
!pip install yfinance
import yfinance as yf
import pandas as pd

# Yahoo Finance APi typically only provides 15-minute interval data for the last 60 days
dataF = yf.download("EURUSD=X", start="2023-12-07", end="2024-02-03", interval='15m')
dataF.iloc[-1:,:]
dataF.Open.iloc

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


/Users/alicehu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[*********************100%%**********************]  1 of 1 completed


## Defining Signal Function

> Do the last few candles have engulfing patterns?

In [2]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]

    # Bearish Pattern
    if (open>close and previous_open<previous_close and close<previous_open and open>=previous_close):
        return 1    # Selling signal
    
    # Bullish Pattern
    elif (open<close and previous_open>previous_close and close>previous_open and open<=previous_close):
        return 2    # Buying signal
    
    # No Clear Pattern
    else:
        return 0
    
signal = []
signal.append(0)
for i in range(1, len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))

#signal_generator(data)
dataF["signal"] = signal

In [3]:
# Market Print Statements

dataF.signal.value_counts()
#dataF.iloc[:, :]

signal
0    3549
2     158
1     148
Name: count, dtype: int64

## Market Connection and Trade Execution

In [4]:
!pip install APScheduler
!pip install oandapyV20
!pip install oanda_candles

from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleCollector, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [5]:
from config import access_token, accountID
#Get the latest candlestick data for EUR/USD pair
def get_candles(n):
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)    # Fetch the latest 3 candles
for candle in candles:
    print(float(str(candle.bid.o))>1)   # Print whether the opening price of the candle is greater than 1

def trading_job():
    # Fetching latest price data
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open', 'Close', 'High', "Low"])

    # Parsing candle data for trading signals
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    # Generating a trading signal
    signal = signal_generator(dfstream.iloc[:-1, :])

    # EXECUTING ORDERS
    client = API(access_token)

    SLTPRatio = 2.
    previous_candleR = abs(dfstream['Open'].iloc[-2]-dfstream['Close'].iloc[-2])

    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio

    print(dfstream.iloc[:-1, :])
    print(TPBuy, " ", SLBuy, " ", TPSell, " ", SLSell)

    # Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-10000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

    # Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=10000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

True
True
True


## Executing Trades Automatically

In [6]:
# MANUAL
#trading_job()

# SCHEDULED
scheduler = BlockingScheduler()
scheduler.add_job(trading_job, 'cron', day_of_week='mon-sun', hour='00-23', minute='0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59', second='18,37,56', start_date='2023-09-26 01:00:00')
scheduler.start()